In [1]:
from metrics import *
import pandas as pd
import os

In [2]:
# MODEL_NAME = 'kaggle_bert'
# MODEL_NAME = 'cnn'
# MODEL_NAME = 'roberta-base-unbiased'
# MODEL_NAME = 'roberta-base-unbiased-small'
MODEL_LIST = ['cnn', 'roberta-base-unbiased-small', 'roberta-base-unbiased','kaggle_bert', 'my_kaggle_bert']

In [3]:
# all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
TOXICITY_COLUMN = 'target'

In [4]:
def convert_to_bool(df, col_name):
        df[col_name] = np.where(df[col_name] >= 0.5, True, False)
        
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns: # no target here
        convert_to_bool(bool_df, col)
    return bool_df

In [5]:
for MODEL_NAME in MODEL_LIST:
    if os.path.exists(f'./bias/{MODEL_NAME}_metrics.csv'):
        bias_metrics_df = pd.read_csv(f'./bias/{MODEL_NAME}_metrics.csv')
        print(f'{MODEL_NAME}: ', get_final_metric(bias_metrics_df, calculate_overall_auc(eval_df, MODEL_NAME)))
    else:    
        eval_df = pd.read_csv(f'./submissions/{MODEL_NAME}_submission.csv')
        if not os.path.exists(f'./submissions/{MODEL_NAME}_submission_plain.csv'):
            submission = pd.DataFrame.from_dict({
            'id': eval_df['id'],
            'prediction': eval_df[MODEL_NAME]
            })
            submission.to_csv(f'./submissions/{MODEL_NAME}_submission_plain.csv', index=False)
        eval_df = convert_dataframe_to_bool(eval_df)
        bias_metrics_df = compute_bias_metrics_for_model(eval_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
        bias_metrics_df.to_csv(f'./bias/{MODEL_NAME}_metrics.csv', index=False)
        print(f'{MODEL_NAME}: ', get_final_metric(bias_metrics_df, calculate_overall_auc(eval_df, MODEL_NAME)))

cnn:  0.8811063685736299
roberta-base-unbiased-small:  0.9336281439245725
roberta-base-unbiased:  0.9374469355554393
kaggle_bert:  0.9383775605488903
my_kaggle_bert:  0.935123053682531
